In [9]:
import pandas as pd
from pymongo import MongoClient
import json

In [10]:
def create_geo_json(row):
    """
    Creates a nested geolocation dictionary from a DataFrame row.
    Returns None if the latitude is NaN (indicating no merge match).
    """
    if pd.isna(row['geolocation_lat']): 
        return None
    return {
        "geolocation_zip_code_prefix": row['geolocation_zip_code_prefix'],
        "geolocation_lat": row['geolocation_lat'],
        "geolocation_lng": row['geolocation_lng'],
        "geolocation_city": row['geolocation_city'],
        "geolocation_state": row['geolocation_state']
    }

# 1. geolocation collection

In [11]:
geolocation = pd.read_csv('olist_geolocation_dataset.csv')

# Prepare the standalone geolocation collection for MongoDB
geolocation_collection = geolocation.to_dict(orient='records')

# Remove duplicates to ensure unique combination of zip, city, and state for merging to customers and sellers
geolocation = geolocation.drop_duplicates(subset=[
    'geolocation_zip_code_prefix', 
    'geolocation_city', 
    'geolocation_state'
])

# 2. customer collection

In [12]:
customers = pd.read_csv('olist_customers_dataset.csv')

# Merge customers with geolocation data based on Zip, City, and State
# Using 'left' join ensures customers are kept even if no geo match is found (simulating dirty data)
merged_customers_geo = customers.merge(
    geolocation, 
    left_on=['customer_zip_code_prefix', 'customer_city', 'customer_state'], 
    right_on=['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'], 
    how='left'
)

# Create nested 'geolocation' field
merged_customers_geo['geolocation'] = merged_customers_geo.apply(create_geo_json, axis=1)

# Drop flat geolocation columns to keep the document structure clean
cols_to_drop = [
    'geolocation_lat', 'geolocation_lng', 
    'geolocation_city', 'geolocation_state', 'geolocation_zip_code_prefix'
]
merged_customers_geo = merged_customers_geo.drop(columns=cols_to_drop)

# Convert to list of dictionaries
customers_collection = merged_customers_geo.to_dict(orient='records')

# 3. sellers collection

In [13]:
sellers = pd.read_csv('olist_sellers_dataset.csv')

# Merge sellers with geolocation data based on Zip, City, and State
merged_sellers_geo = sellers.merge(
    geolocation, 
    left_on=['seller_zip_code_prefix', 'seller_city', 'seller_state'], 
    right_on=['geolocation_zip_code_prefix', 'geolocation_city', 'geolocation_state'], 
    how='left'
)

# Create nested 'geolocation' field
merged_sellers_geo['geolocation'] = merged_sellers_geo.apply(create_geo_json, axis=1)

# Drop flat geolocation columns
merged_sellers_geo = merged_sellers_geo.drop(columns=cols_to_drop)

# Convert to list of dictionaries
sellers_collection = merged_sellers_geo.to_dict(orient='records')

# 4. product collection

In [14]:
products = pd.read_csv('olist_products_dataset.csv')
product_category_translation = pd.read_csv('product_category_name_translation.csv')

# Normalize for better matching
products['product_category_name'] = products['product_category_name'].str.lower().str.strip()
product_category_translation['product_category_name'] = product_category_translation['product_category_name'].str.lower().str.strip()

# Merge products with translations to get English category names
merged_products = products.merge(product_category_translation, on='product_category_name', how='left')

# Convert to list of dictionaries
products_collection = merged_products.to_dict(orient='records')

# 5. order collection

In [22]:
orders_df = pd.read_csv('olist_orders_dataset.csv')
items_df = pd.read_csv('olist_order_items_dataset.csv')
payments_df = pd.read_csv('olist_order_payments_dataset.csv')
reviews_df = pd.read_csv('olist_order_reviews_dataset.csv')

# --- Grouping Items ---
# Group items by order_id and convert to list of dictionaries
# Drop 'order_id' from the nested objects to avoid redundancy
items_grp = items_df.groupby('order_id').apply(
    lambda x: x.drop('order_id', axis=1).to_dict('records')
).reset_index(name='order_items')

# --- Grouping Payments ---
payments_grp = payments_df.groupby('order_id').apply(
    lambda x: x.drop('order_id', axis=1).to_dict('records')
).reset_index(name='order_payments')

# --- Grouping Reviews ---
reviews_grp = reviews_df.groupby('order_id').apply(
    lambda x: x.drop('order_id', axis=1).to_dict('records')
).reset_index(name='order_reviews')

# --- Merging into Main Orders DataFrame ---
# Left join to ensure all orders are kept, even if they have no items/payments/reviews
orders_final = orders_df.merge(items_grp, on='order_id', how='left')
orders_final = orders_final.merge(payments_grp, on='order_id', how='left')
orders_final = orders_final.merge(reviews_grp, on='order_id', how='left')

# --- Handling Missing Nested Data ---
# If an order has no items/payments, the merge results in NaN. 
# We replace NaN with empty lists [] for consistency in MongoDB.
for col in ['order_items', 'order_payments', 'order_reviews']:
    orders_final[col] = orders_final[col].apply(lambda x: x if isinstance(x, list) else [])

# --- Converting Date Columns ---
# Convert string dates to Python datetime objects for proper storage in MongoDB
date_cols = [
    'order_purchase_timestamp', 'order_approved_at', 
    'order_delivered_carrier_date', 'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]
for col in date_cols:
    orders_final[col] = pd.to_datetime(orders_final[col])
    orders_final[col] = orders_final[col].astype(object).where(orders_final[col].notnull(), None)

# Convert to list of dictionaries
orders_collection = orders_final.to_dict(orient='records')

C:\Users\vuloc\AppData\Local\Temp\ipykernel_17896\203589426.py:9: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  items_grp = items_df.groupby('order_id').apply(
C:\Users\vuloc\AppData\Local\Temp\ipykernel_17896\203589426.py:14: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  payments_grp = payments_df.groupby('order_id').apply(
C:\Users\vuloc\AppData\Local\Temp\ipykernel_17896\203589426.py:19: FutureWarning: DataFrameGr

# 6. insert to mongodb

In [ ]:
# Connect to local MongoDB instance
# Replace connection string if using a cloud database (e.g., MongoDB Atlas)
client = MongoClient('')

# Create (or access) the database for the Legacy System
db = client['olist_legacy_system']
print(client)

MongoClient(host=['ac-mh9db4u-shard-00-02.a04msnj.mongodb.net:27017', 'ac-mh9db4u-shard-00-00.a04msnj.mongodb.net:27017', 'ac-mh9db4u-shard-00-01.a04msnj.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, appname='Cluster0', authsource='admin', replicaset='atlas-7va5io-shard-0', tls=True)


In [24]:
# Clear existing data to avoid duplicates during testing (Optional)
db.geolocation.drop()
db.customers.drop()
db.sellers.drop()
db.products.drop()
db.orders.drop()

# Insert data into collections
# Using insert_many for batch insertion which is faster
db.geolocation.insert_many(geolocation_collection)
db.customers.insert_many(customers_collection)
db.sellers.insert_many(sellers_collection)
db.products.insert_many(products_collection)
db.orders.insert_many(orders_collection)

# Verification message
print("Successfully migrated Olist dataset to MongoDB!")

Successfully migrated Olist dataset to MongoDB!
